# 1 Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import torch

import random
#import nlpaug.augmenter.word as naw
import json

import nltk
from nltk.corpus import wordnet

In [ ]:
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nicol\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\nicol\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.


True

# 2 Loading dataset

In [3]:
fine_tuning_data_set = pd.read_excel('C:/Users/nicol/OneDrive/OPER/2025/Gemini_persona/dados/df_fine_tuning.xlsx')

# 3 Data Augmentation

Reduz overfitting; 
Melhor generalização;
Maior diversidade.

In [4]:
fine_tuning_data_set[(fine_tuning_data_set['Q_label'] == 'Q23')]['answer'].unique()

array([800, 500, 750, 200, 850, 250, 600, 900], dtype=object)

In [5]:
fine_tuning_data_set[(fine_tuning_data_set['Q_label'] == 'Q20') & (fine_tuning_data_set['answer'] == 500 )].head()

,Unnamed: 0,person,Question,answer,Q_label,Kmeans_pca
236,25354,"Man,18-24,Bachelor's Degree (undergraduate),< ...",Imagine that you and three other colleagues on...,500,Q20,3
1186,25388,"Man,18-24,PhD / Doctorate,< 1000 USD/month (lo...",Imagine that you and three other colleagues on...,500,Q20,0
1683,25406,"Man,25-34,High School or equivalent,1000-2999 ...",Imagine that you and three other colleagues on...,500,Q20,3
8387,25645,"Man,65+,Other / Non-traditional education,1000...",Imagine that you and three other colleagues on...,500,Q20,3
9501,25685,"Non-binary,18-24,High School or equivalent,500...",Imagine that you and three other colleagues on...,500,Q20,3


In [6]:
def generate_expansive_response_scale(question_id, value):
    """
    Generates coherent explanations for numeric/text responses.
    
    Parameters:
        question_id (str): 'Q1' to 'Q28'
        value (int or float): Numeric answer or score
    Returns:
        str: Expanded, human-readable explanation
    """
    
    # Perguntas Q1-Q19, Q27-Q28 (1 a 6)
    standard_questions = [
        'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9',
        'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q27', 'Q28'
    ]
    
    if question_id in standard_questions:
        if isinstance(value, int) and 1 <= value <= 6:
            # Low score
            if value <= 2:
                intensity = random.choice([
                    "shows low alignment with the described behavior",
                    "indicates a tendency not to see themselves this way",
                    "reflects limited identification with this trait"
                ])
            # Medium score
            elif value in [3, 4]:
                intensity = random.choice([
                    "indicates moderate alignment with the described behavior",
                    "reflects a balanced view between agreement and disagreement",
                    "suggests a neutral or ambivalent stance"
                ])
            # High score
            else:
                intensity = random.choice([
                    "demonstrates strong alignment with the described behavior",
                    "shows high affinity for this characteristic",
                    "indicates a clear tendency to see themselves this way"
                ])
            return f"{value} {intensity}."
    

In [7]:
def generate_expansive_response_monetary(question_id, value):
    """
    Generates coherent explanations for numeric/text responses.
    
    Parameters:
        question_id (str): 'Q20' to 'Q23'
        value (int or float): Numeric answer or score
    Returns:
        str: Expanded, human-readable explanation
    """

    
    if question_id == "Q20":
        if isinstance(value, int) and 0 <= value <= 2000:
            
            if value in [0,499]:
                intensity = random.choice([
                    "I prefer not to invest in the project, keeping all of my bonus for myself.",
                    "I contribute with nothing, prioritizing personal gain over team benefit.",
                    "I decide to invest nothing, focusing on my own outcome."
                ])
            elif value in [500, 999]:
                intensity = random.choice([
                    "I contribute a small portion, showing some willingness to support the team.",
                    "I invest a moderate amount, balancing personal gain and team benefit.",
                    "I put in a modest amount, demonstrating limited cooperation."
                ])

            elif value in [1000, 1499]:
                intensity = random.choice([
                        "I invest around half, balancing personal gain and team improvement.",
                        "I contribute a significant portion, showing moderate trust in colleagues.",
                        "I split my bonus to equally value my own and the team's benefit."
                ])
            
            elif value in [1500, 1999]:
                intensity = random.choice([
                        "I invest most of my bonus, prioritizing team benefit over personal gain.",
                        "I contribute a large amount, showing strong cooperation and trust in the team.",
                        "I put in a significant sum, valuing collective success."
                ])

            
            else:
                intensity = random.choice([
                        "I fully invest my bonus, showing complete trust and commitment to the team's success.",
                        "I contribute everything, prioritizing the common good.",
                        "I invest the maximum, demonstrating maximum collaboration and confidence in colleagues."
                ])
            return f"Invested US${value}, {intensity}"
        

    if question_id == "Q21":
        if isinstance(value, int) and 0 <= value <= 1000:
            if 0 <= value <= 99:
                intensity = random.choice([
                    "I keep the entire amount, prioritizing my own gain over fairness.",
                    "I prefer to retain all the money, showing little concern for the colleagues share.",
                    "I decide not to give anything, focusing solely on my own outcome."
                ])
            elif 100 <= value <= 299:
                intensity = random.choice([
                    "I give a small portion, showing minimal generosity.",
                    "I share a little, but still prioritize myself over the colleague.",
                    "I offer a small amount, reflecting limited fairness."
                ])
            elif 300 <= value <= 599:
                intensity = random.choice([
                    "I offer a fair share, balancing self-interest with a sense of justice.",
                    "I split the money moderately, showing balanced generosity.",
                    "I choose a middle ground, demonstrating fairness without full equality."
                ])
            elif 600 <= value <= 899:
                intensity = random.choice([
                    "I offer a large portion, showing strong fairness and generosity.",
                    "I prioritize equality, giving the colleague almost as much as myself.",
                    "I value fairness highly, sharing most of the bonus."
                ])
            else:  # 900–1000
                intensity = random.choice([
                    "I give nearly everything, showing complete fairness and altruism.",
                    "I prioritize my colleague entirely, demonstrating extreme generosity.",
                    "I share the maximum possible, fully trusting and valuing fairness."
                ])
            return f"Gaved US${value}, {intensity}"

    # Q22 – Proposal fairness under acceptance/rejection (0–1000)
    if question_id == "Q22":
        if isinstance(value, int) and 0 <= value <= 1000:
            if 0 <= value <= 99:
                intensity = random.choice([
                    "I propose giving almost nothing, prioritizing personal profit over fairness.",
                    "I make a selfish proposal, expecting the colleague to accept minimal gain.",
                    "I offer an unfair split, assuming power in negotiation."
                ])
            elif 100 <= value <= 299:
                intensity = random.choice([
                    "I offer a small amount, testing how much I can keep while avoiding rejection.",
                    "I share minimally, prioritizing self-interest with some awareness of fairness.",
                    "I give a limited offer, aiming to keep most of the money."
                ])
            elif 300 <= value <= 599:
                intensity = random.choice([
                    "I propose a balanced division, showing moderate fairness and strategy.",
                    "I make a fair offer that is likely to be accepted.",
                    "I split the amount reasonably, balancing fairness with self-interest."
                ])
            elif 600 <= value <= 899:
                intensity = random.choice([
                    "I make a generous proposal, showing strong fairness and empathy.",
                    "I offer most of the money, prioritizing a good outcome for both.",
                    "I value fairness highly, proposing a near-equal split."
                ])
            else:  # 900–1000
                intensity = random.choice([
                    "I propose giving nearly everything, showing extreme generosity.",
                    "I prioritize fairness completely, putting the colleague’s gain above my own.",
                    "I demonstrate full altruism, offering the maximum possible amount."
                ])
            return f"Gaved US${value}, {intensity}"

    # Q23 – Minimum acceptable amount (0–1000)
    if question_id == "Q23":
        if isinstance(value, int) and 0 <= value <= 1000:
            if 0 <= value <= 99:
                intensity = random.choice([
                    "I would accept almost any offer, showing very low concern for fairness.",
                    "I am willing to accept minimal gains, indicating high tolerance for inequality.",
                    "I have very low expectations, accepting even unfair offers."
                ])
            elif 100 <= value <= 299:
                intensity = random.choice([
                    "I would accept small offers, showing limited insistence on fairness.",
                    "I am modest in my expectations, accepting somewhat low amounts.",
                    "I demonstrate flexibility by accepting less than equal splits."
                ])
            elif 300 <= value <= 700:
                intensity = random.choice([
                    "I expect a fair portion, showing balanced concern for fairness and realism.",
                    "I would accept a moderately fair offer, neither too generous nor too strict.",
                    "I show a reasonable expectation for equitable treatment."
                ])
            elif 701 <= value <= 899:
                intensity = random.choice([
                    "I would only accept a high offer.",
                    "I expect substantial compensation, valuing my own gain.",
                    "I demonstrate assertiveness in ensuring a big part of the amount"
                ])
            else:  # 900–1000
                intensity = random.choice([
                    "I would only accept almost all the amount, rejecting anything else.",
                    "I show very high standards, demanding almost all the amount.",
                    "I am firm in expecting  full amount , prioritizing my personal gain."
                ])
            return f"Would accepted US${value}, {intensity}"

    

In [8]:
fine_tuning_data_set.drop(columns= ["Unnamed: 0"], inplace=True)

In [9]:
fine_tuning_data_set[fine_tuning_data_set["Q_label"].isin(['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9',
       'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q27', 'Q28'])]["answer"].unique()

array([5, 4, 3, 1, 2, 6], dtype=object)

In [10]:
fine_tuning_data_set[fine_tuning_data_set["Q_label"].isin(['Q20', 'Q21', 'Q23'])]["answer"].unique()

array([800, 600, 2000, 1500, 0, 500, 750, 1000, 200, 250, 300, 850, 700,
       400, 50, 1800, 100, 900], dtype=object)

In [11]:
fine_tuning_data_set['expanded_answer'] = fine_tuning_data_set.apply(
    lambda row: (
        generate_expansive_response_scale(row['Q_label'], row['answer'])
        if row['Q_label'] in [
            'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9',
            'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q27', 'Q28'
        ] else (
            generate_expansive_response_monetary(row['Q_label'], row['answer'])
            if row['Q_label'] in ['Q20', 'Q21', 'Q22', 'Q23']
            else row['answer']
        )
    ),
    axis=1
)

In [12]:
len(fine_tuning_data_set[fine_tuning_data_set["Q_label"].isin(['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9',
       'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q27', 'Q28'])]["expanded_answer"].unique())

18

In [13]:
len(fine_tuning_data_set[fine_tuning_data_set["Q_label"].isin(['Q20', 'Q21', 'Q23'])]["expanded_answer"].unique())

65

In [14]:
fine_tuning_data_set['answer'].nunique(), fine_tuning_data_set['expanded_answer'].nunique()

(402, 481)

# 4 Data formatting to gemini fine tuning

In [15]:
def write_jsonl_file(cluster_profile, dataset, output_file_name):
    """
    Create jsonl file for Gemini fine-tuning format.
    """
    with open(output_file_name, 'w', encoding='utf-8') as f:
        print(f"Criando o arquivo {output_file_name}...")

        for index, row in dataset.iterrows():
            question = row['Question']
            expected_answer = row['expanded_answer']

            json_line_data = {
                "contents": [
                    {
                        "role": "user",
                        "parts": [
                            {"text": f"{cluster_profile}\n\nQuestion: {question}"}
                        ]
                    },
                    {
                        "role": "model",
                        "parts": [
                            {"text": expected_answer}
                        ]
                    }
                ]
            }

            f.write(json.dumps(json_line_data, ensure_ascii=False) + "\n")

    print("Arquivo .jsonl created!")


In [16]:
def format_data(cluster_profile, dataset, cluster_number, cluster_name):
    """
    Format the data to each cluster
    """
    # Selecting only the data from the clsuter
    dataset = dataset[dataset["Kmeans_pca"] == cluster_number].copy()

    # Getting all unique people in the dataset
    all_people_list = list(dataset['person'].unique())
    
    # Separation in train and validation sets
    train_people_list = random.sample(all_people_list, np.ceil(len(all_people_list)*0.8).astype(int))
    validation_people_list = [p for p in all_people_list if p not in train_people_list]

    train_dataset = dataset[dataset['person'].isin(train_people_list)]
    validation_dataset = dataset[dataset['person'].isin(validation_people_list)]


    output_file_name_validation = f'C:/Users/nicol/OneDrive/OPER/2025/Gemini_persona/json/validation_cluster_{cluster_name}_gemini.jsonl'
    output_file_name_train = f'C:/Users/nicol/OneDrive/OPER/2025/Gemini_persona/json/train_cluster_{cluster_name}_gemini.jsonl'

    write_jsonl_file(cluster_profile, validation_dataset, output_file_name_validation)
    write_jsonl_file(cluster_profile, train_dataset, output_file_name_train)

    print(f"Finished processing cluster {cluster_name}.")
    print(f"Train samples: {len(train_dataset)}, Validation samples: {len(validation_dataset)}")

## Cluster Profiles

In [17]:
# Cluster 0
Pragmatic_Guardian = """Is composed of individuals who combine a concern for personal security and benevolence with a strong preference for stability and job security. 
                            They exhibit cooperative behavior (contributing substantially to the common good and being relatively generous when sharing money), but they also demand a reasonable return when others make proposals. 
                            There is a paradox: despite risk aversion in abstract beliefs, in concrete choices, they accept moderate risk—suggesting pragmatic and conditional behavior: they take risks when the reward is attractive."""

# Cluster 1
Ambitious_Innovator = """Is the most creative, curious people who enjoy challenges, but also possess a strong sense of sacrifice for the future (a willingness to invest in long-term goals) with strong preferences for a competitive salary and opportunities for rapid career growth. 
                          Socially, they show moderate altruism and prefer environments that support work-life balance and clear feedback. 
                          Financially, they contribute high percentages to collective projects (high cooperativism) and accept risk when a potential return exists."""

# CLuster 2
Disciplined_Traditionalist = """Is conservative and disciplined: they value security, rules, meritocracy, and the stability of institutions. 
                          Paradoxically, in practical monetary contexts, they show a great willingness to cooperate and share resources (high contributions and donations in the scenarios), which indicates a high trust in the norms of reciprocity (they believe that cooperation pays off). 
                          They prioritize competitive salary and job stability and security, maintaining a conservative bias regarding novelty and risks."""

# CLuster 3
Security_Seeker = """Is the most risk-averse and least open to novelty and challenge. 
                          They prefer stable and predictable environments, and they choose the safe option when confronted with probabilities. 
                          The strong value placed on job stability and security, along with a strong emphasis on work-life balance, reinforces their preference for safe options and an aversion to experimental environments. 
                          Financially, they demonstrate defensive and self-interested behavior in resource-sharing games."""

In [18]:
format_data(Pragmatic_Guardian, fine_tuning_data_set, 0, "pragmatic_guardian")

Criando o arquivo C:/Users/nicol/OneDrive/OPER/2025/Gemini_persona/json/validation_cluster_pragmatic_guardian_gemini.jsonl...
Arquivo .jsonl created!
Criando o arquivo C:/Users/nicol/OneDrive/OPER/2025/Gemini_persona/json/train_cluster_pragmatic_guardian_gemini.jsonl...
Arquivo .jsonl created!
Finished processing cluster pragmatic_guardian.
Train samples: 9352, Validation samples: 2324


In [19]:
format_data(Ambitious_Innovator, fine_tuning_data_set, 1, "ambitious_innovator")

Criando o arquivo C:/Users/nicol/OneDrive/OPER/2025/Gemini_persona/json/validation_cluster_ambitious_innovator_gemini.jsonl...
Arquivo .jsonl created!
Criando o arquivo C:/Users/nicol/OneDrive/OPER/2025/Gemini_persona/json/train_cluster_ambitious_innovator_gemini.jsonl...
Arquivo .jsonl created!
Finished processing cluster ambitious_innovator.
Train samples: 9604, Validation samples: 2380


In [20]:
format_data(Disciplined_Traditionalist, fine_tuning_data_set, 2, "disciplined_traditionalist")

Criando o arquivo C:/Users/nicol/OneDrive/OPER/2025/Gemini_persona/json/validation_cluster_disciplined_traditionalist_gemini.jsonl...
Arquivo .jsonl created!
Criando o arquivo C:/Users/nicol/OneDrive/OPER/2025/Gemini_persona/json/train_cluster_disciplined_traditionalist_gemini.jsonl...
Arquivo .jsonl created!
Finished processing cluster disciplined_traditionalist.
Train samples: 6496, Validation samples: 1624


In [21]:
format_data(Security_Seeker, fine_tuning_data_set, 3, "security_seeker")

Criando o arquivo C:/Users/nicol/OneDrive/OPER/2025/Gemini_persona/json/validation_cluster_security_seeker_gemini.jsonl...
Arquivo .jsonl created!
Criando o arquivo C:/Users/nicol/OneDrive/OPER/2025/Gemini_persona/json/train_cluster_security_seeker_gemini.jsonl...
Arquivo .jsonl created!
Finished processing cluster security_seeker.
Train samples: 4480, Validation samples: 1092


In [37]:
def read_jsonl_file(filepath):
    """
    Reads a JSONL file and returns a list of Python dictionaries,
    where each dictionary represents a JSON object from a line.
    """
    data = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                json_object = json.loads(line.strip())
                data.append(json_object)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON on line: {line.strip()} - {e}")
    return data

# Example usage:
file_path = 'C:/Users/nicol/OneDrive/OPER/2025/Gemini_persona/json/validation_cluster_disciplined_traditionalist_gemini.jsonl'  # Replace with the actual path to your JSONL file
jsonl_data = read_jsonl_file(file_path)

In [38]:
jsonl_data

[{'contents': [{'role': 'user',
    'parts': [{'text': 'Is conservative and disciplined: they value security, rules, meritocracy, and the stability of institutions. \n                          Paradoxically, in practical monetary contexts, they show a great willingness to cooperate and share resources (high contributions and donations in the scenarios), which indicates a high trust in the norms of reciprocity (they believe that cooperation pays off). \n                          They prioritize competitive salary and job stability and security, maintaining a conservative bias regarding novelty and risks.\n\nQuestion: You will be asked to say How much is this person like you in a scale of 1 to 6, where 1 is “Nothing like me” and 6 is “A lot like me”. \n    They want to be always doing new things, even though some of them might hard and challenging.'}]},
   {'role': 'model',
    'parts': [{'text': '4 suggests a neutral or ambivalent stance.'}]}]},
 {'contents': [{'role': 'user',
    'part